In [1]:
from pm4py.objects.log.importer.xes import importer as xes_importer
import pm4py
from collections import Counter
import cProfile
import inspect, pulp
print([name for name, obj in inspect.getmembers(pulp) if "GUROBI" in name])





Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
['GUROBI', 'GUROBI_CMD']


In [2]:
log_path_2012 = 'BPI_Challenge/BPI_Challenge_2017.xes'
log_2012 = xes_importer.apply(log_path_2012)

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|███| 31509/31509 [00:31<00:00, 988.43it/s]


In [3]:
def _convert_log_to_counter(log):
    """Convert any log format to Counter format"""
    if isinstance(log, Counter):
        return log
    
    freq_dict = Counter()
    
    # Handle PM4Py EventLog object
    if hasattr(log, '__class__') and 'EventLog' in str(log.__class__):
        for trace in log:
            activities = tuple(event['concept:name'] for event in trace)
            freq_dict[activities] += 1
    
    # Handle list of traces
    elif isinstance(log, list):
        for trace in log:
            if trace and isinstance(trace[0], dict):
                activities = tuple(event['concept:name'] for event in trace)
            else:
                activities = tuple(trace)
            freq_dict[activities] += 1
    
    else:
        raise ValueError(f"Unsupported log type: {type(log)}")
    
    return freq_dict

In [4]:
log_counter = _convert_log_to_counter(log_2012)
type(log_counter)

collections.Counter

In [5]:
print("traces: ",len(log_counter),"cases: ", sum(log_counter.values()))

traces:  15930 cases:  31509


In [6]:
from collections import Counter

def remove_w_stuff(log_counter):

    cleaned_counter = Counter()
    for trace, count in log_counter.items():
        cleaned_trace = tuple(event for event in trace if not event.startswith('W_'))
        cleaned_counter[cleaned_trace] += count

    return cleaned_counter

In [7]:
clean_log_counter = remove_w_stuff(log_counter)
print("traces: ",len(clean_log_counter),"cases: ", sum(clean_log_counter.values()))

traces:  2630 cases:  31509


In [8]:
import time
from pm4py.objects.log import obj as log_instance

In [9]:
def remove_incomplete_traces(log_counter, end_activities_list):
    completed_counter = Counter()
    for trace, count in log_counter.items():
        if trace[-1] in end_activities_list:
            completed_counter[trace] +=count
    return completed_counter

In [10]:
end_activities_list_2017  = [
    'A_Pending',
    'A_Denied',
    'A_Cancelled'
]
end_activities_list_2012 = ['A_DECLINED', 'A_CANCELLED', 'A_REGISTERED', 'A_APPROVED', 'A_ACTIVATED', 'O_ACCEPTED', 'O_DECLINED', 'O_CANCELLED']
clean_complete_log_counter = remove_incomplete_traces(clean_log_counter,end_activities_list_2017)
print("traces: ",len(clean_complete_log_counter),"cases: ", sum(clean_complete_log_counter.values()))

logM = log_instance.EventLog()
logM.append(log_instance.Trace())

from local_pm4py import discovery

start = time.time()
#net, initial_marking, final_marking, process_tree, sorted_cuts = discovery.apply_bi(clean_log_counter, logM)
end = time.time()
print("run time:")
print(end-start)
#print(process_tree)

traces:  419 cases:  12986
run time:
1.2874603271484375e-05


In [11]:
from IM_ilp.recursion_main import recursion_full as recursion_full
from IM_ilp.recursion_exp import recursion_full as recursion_full_improved
import time

In [12]:
start_for_stats_0 = time.time()
process_tree_exp_0 = recursion_full_improved(clean_log_counter, sup =0)
end_for_stats_0 = time.time()



=== Recursion Depth 0 ===
__
Log has 31509 total trace occurrences
Log has 2630 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'A_Denied', 'O_Refused'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Cancelled', 'O_Sent (mail and online)', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending')]
Unique activities: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Applicatio

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Application', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Accepted', 'A_Cancelled', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Refused', 'A_Denied'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Application', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', '

...

  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Incomplete', 'A_Validating', 'O_Sent (mail and online)', 'O_Create Offer'}, Sigma_2={'A_Pending', 'O_Cancelled', 'A_Cancelled', 'O_Accepted'}, cost=1467
  ✓ seq cut successful with cost 1467 (new best)
Trying par cut...
ILP failed with status: Not Solved
  par cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ par cut invalid - empty sets or infinite cost
Trying loop cut...


/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


Cost approximated
  loop cut result: Sigma_1={'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, Sigma_2={'O_Accepted'}, cost=1.0
  ✓ loop cut successful with cost 1.0 (new best)
Best cut: loop with cost 1.0
Splitting log with loop cut...
  Sigma_1: {'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}
  Sigma_2: {'O_Accepted'}
Split results:
  Log1: 1984 unique traces, 48737 total occurrences
  Log2: 1 unique traces, 17228 total occurrences

=== Recursion Depth 3 ===
__
Log has 48737 total trace occurrences
Log has 1984 unique traces
Sample traces: [('O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating'), ('A_Pending',), ('O_Create Offer', 'O

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


ILP failed with status: Not Solved
  par cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ par cut invalid - empty sets or infinite cost
Trying loop cut...
Cost approximated
  loop cut result: Sigma_1={'A_Denied'}, Sigma_2={'O_Refused'}, cost=1.0
  ✓ loop cut successful with cost 1.0
Best cut: seq with cost 1
Splitting log with seq cut...
  Sigma_1: {'A_Denied'}
  Sigma_2: {'O_Refused'}
Split results:
  Log1: 1 unique traces, 3752 total occurrences
  Log2: 9 unique traces, 3752 total occurrences

=== Recursion Depth 2 ===
__
Log has 3752 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Denied',)]
Unique activities: {'A_Denied'}
Base case: Single activity 'A_Denied' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
τ-xor condition satisfied with cost 0
  Sigma_1: {'A_Denied'}, Sigma_2: set()
τ-xor split results:
  Log1: 1 unique traces, 3752 total occurrences
  Log2: 0 unique traces, 0 total occurrences

=== Recurs

In [13]:
print(end_for_stats_0-start_for_stats_0)

8.297553062438965


In [14]:
start_for_stats_01 = time.time()
process_tree_exp_01 = recursion_full_improved(clean_log_counter, sup =0.1)
end_for_stats_01 = time.time()
cProfile.run('recursion_full_improved(clean_log_counter, sup =0.1)')


=== Recursion Depth 0 ===
__
Log has 31509 total trace occurrences
Log has 2630 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'A_Denied', 'O_Refused'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Cancelled', 'O_Sent (mail and online)', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending')]
Unique activities: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Applicatio

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending',

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Submitted', 'A_Concept'}, Sigma_2={'A_Accepted'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'A_Accepted'}
Split results:
  Log1: 2 unique traces, 31509 total occurrences
  Log2: 1 unique traces, 31509 total occurrences

=== Recursion Depth 2 ===
__
Log has 31509 total trace occurrences
Log has 2 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept'), ('A_Create Application', 'A_Concept')]
Unique activities: {'A_Create Application', 

...

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warning

No special structure found, returning single activity leaf node

=== Recursion Depth 9 ===
__
Log has 23240 total trace occurrences
Log has 2 unique traces
Sample traces: [('O_Returned',), ('O_Returned', 'O_Returned')]
Unique activities: {'O_Returned'}
Base case: Single activity 'O_Returned' detected
Graph created with 3 nodes and 3 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 7 ===
__
Log has 42995 total trace occurrences
Log has 1 unique traces
Sample traces: [('O_Create Offer',)]
Unique activities: {'O_Create Offer'}
Base case: Single activity 'O_Create Offer' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 6 ===
__
Log has 46216 total trace occurrences
Log 

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')



=== Recursion Depth 5 ===
__
Log has 10431 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Cancelled',)]
Unique activities: {'A_Cancelled'}
Base case: Single activity 'A_Cancelled' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 4 ===
__
Log has 17228 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Pending',)]
Unique activities: {'A_Pending'}
Base case: Single activity 'A_Pending' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 3 ===
__
Log has 3720 total trace occurrences
Log has 8 unique traces
Sample traces: [('O_Refused',), ('O_Refused', 'O_Refused', 'O_Refused'), ('O_Refused', 

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


In [15]:
#tree_data_pm4py_exp_01 = to_pm4py_tree(process_tree_exp_01)
#net_exp_01, im_exp_01, fm_exp_01 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_01)
#petri_nets = [(net_exp_01, im_exp_01, fm_exp_01)]
#save_multiple_petri_nets_to_pdf(petri_nets, "IM_ilp/model_try.pdf")
#print(end_for_stats_01- start_for_stats_01)

In [16]:
start_for_stats_02 = time.time()
process_tree_exp_02 = recursion_full_improved(clean_log_counter, sup =0.2)
end_for_stats_02 = time.time()



=== Recursion Depth 0 ===
__
Log has 31509 total trace occurrences
Log has 2630 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'A_Denied', 'O_Refused'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Cancelled', 'O_Sent (mail and online)', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending')]
Unique activities: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Applicatio

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending',

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Submitted', 'A_Concept'}, Sigma_2={'A_Accepted'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'A_Accepted'}
Split results:
  Log1: 2 unique traces, 31509 total occurrences
  Log2: 1 unique traces, 31509 total occurrences

=== Recursion Depth 2 ===
__
Log has 31509 total trace occurrences
Log has 2 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept'), ('A_Create Application', 'A_Concept')]
Unique activities: {'A_Create Application', 

...

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warning

No special structure found, returning single activity leaf node

=== Recursion Depth 6 ===
__
Log has 17228 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Pending',)]
Unique activities: {'A_Pending'}
Base case: Single activity 'A_Pending' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 5 ===
__
Log has 10431 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Cancelled',)]
Unique activities: {'A_Cancelled'}
Base case: Single activity 'A_Cancelled' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 4 ===
__
Log has 3753 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Den

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


No special structure found, returning single activity leaf node

=== Recursion Depth 3 ===
__
Log has 3720 total trace occurrences
Log has 8 unique traces
Sample traces: [('O_Refused',), ('O_Refused', 'O_Refused', 'O_Refused'), ('O_Refused', 'O_Refused')]
Unique activities: {'O_Refused'}
Base case: Single activity 'O_Refused' detected
Graph created with 3 nodes and 3 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 2 ===
__
Log has 17228 total trace occurrences
Log has 1 unique traces
Sample traces: [('O_Accepted',)]
Unique activities: {'O_Accepted'}
Base case: Single activity 'O_Accepted' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node


/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


In [17]:
start_for_stats_03 = time.time()
process_tree_exp_03 = recursion_full_improved(clean_log_counter, sup =0.3)
end_for_stats_03 = time.time()



=== Recursion Depth 0 ===
__
Log has 31509 total trace occurrences
Log has 2630 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'A_Denied', 'O_Refused'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Cancelled', 'O_Sent (mail and online)', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending')]
Unique activities: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Applicatio

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending',

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Submitted', 'A_Concept'}, Sigma_2={'A_Accepted'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'A_Accepted'}
Split results:
  Log1: 2 unique traces, 31509 total occurrences
  Log2: 1 unique traces, 31509 total occurrences

=== Recursion Depth 2 ===
__
Log has 31509 total trace occurrences
Log has 2 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept'), ('A_Create Application', 'A_Concept')]
Unique activities: {'A_Create Application', 

...

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warning

No special structure found, returning single activity leaf node

=== Recursion Depth 6 ===
__
Log has 10431 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Cancelled',)]
Unique activities: {'A_Cancelled'}
Base case: Single activity 'A_Cancelled' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 5 ===
__
Log has 42995 total trace occurrences
Log has 1 unique traces
Sample traces: [('O_Create Offer',)]
Unique activities: {'O_Create Offer'}
Base case: Single activity 'O_Create Offer' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 4 ===
__
Log has 3753 total trace occurrences
Log has 1 unique traces
Sample t

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')



=== Recursion Depth 3 ===
__
Log has 3720 total trace occurrences
Log has 8 unique traces
Sample traces: [('O_Refused',), ('O_Refused', 'O_Refused', 'O_Refused'), ('O_Refused', 'O_Refused')]
Unique activities: {'O_Refused'}
Base case: Single activity 'O_Refused' detected
Graph created with 3 nodes and 3 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 2 ===
__
Log has 17228 total trace occurrences
Log has 1 unique traces
Sample traces: [('O_Accepted',)]
Unique activities: {'O_Accepted'}
Base case: Single activity 'O_Accepted' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node


/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


In [18]:
start_for_stats_04 = time.time()
process_tree_exp_04 = recursion_full_improved(clean_log_counter, sup =0.4)
end_for_stats_04 = time.time()



=== Recursion Depth 0 ===
__
Log has 31509 total trace occurrences
Log has 2630 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'A_Denied', 'O_Refused'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Cancelled', 'O_Sent (mail and online)', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending')]
Unique activities: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Applicatio

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending',

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Submitted', 'A_Concept'}, Sigma_2={'A_Accepted'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'A_Accepted'}
Split results:
  Log1: 2 unique traces, 31509 total occurrences
  Log2: 1 unique traces, 31509 total occurrences

=== Recursion Depth 2 ===
__
Log has 31509 total trace occurrences
Log has 2 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept'), ('A_Create Application', 'A_Concept')]
Unique activities: {'A_Create Application', 

...

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warning

No special structure found, returning single activity leaf node

=== Recursion Depth 10 ===
__
Log has 31362 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Complete',)]
Unique activities: {'A_Complete'}
Base case: Single activity 'A_Complete' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 9 ===
__
Log has 23240 total trace occurrences
Log has 2 unique traces
Sample traces: [('O_Returned',), ('O_Returned', 'O_Returned')]
Unique activities: {'O_Returned'}
Base case: Single activity 'O_Returned' detected
Graph created with 3 nodes and 3 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 7 ===
__
Log has 17228 total trace occurrences
Log has 1 uniqu

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


No special structure found, returning single activity leaf node

=== Recursion Depth 5 ===
__
Log has 42995 total trace occurrences
Log has 1 unique traces
Sample traces: [('O_Create Offer',)]
Unique activities: {'O_Create Offer'}
Base case: Single activity 'O_Create Offer' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 4 ===
__
Log has 3753 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Denied',)]
Unique activities: {'A_Denied'}
Base case: Single activity 'A_Denied' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 3 ===
__
Log has 3720 total trace occurrences
Log has 8 unique traces
Sample traces: [('

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


In [19]:
start_for_stats_05 = time.time()
process_tree_exp_05 = recursion_full_improved(clean_log_counter, sup =0.5)
end_for_stats_05 = time.time()



=== Recursion Depth 0 ===
__
Log has 31509 total trace occurrences
Log has 2630 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'A_Denied', 'O_Refused'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Cancelled', 'O_Sent (mail and online)', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending')]
Unique activities: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Applicatio

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending',

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Submitted', 'A_Concept'}, Sigma_2={'A_Accepted'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'A_Accepted'}
Split results:
  Log1: 2 unique traces, 31509 total occurrences
  Log2: 1 unique traces, 31509 total occurrences

=== Recursion Depth 2 ===
__
Log has 31509 total trace occurrences
Log has 2 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept'), ('A_Create Application', 'A_Concept')]
Unique activities: {'A_Create Application', 

...

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warning

No special structure found, returning single activity leaf node

=== Recursion Depth 10 ===
__
Log has 31362 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Complete',)]
Unique activities: {'A_Complete'}
Base case: Single activity 'A_Complete' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 8 ===
__
Log has 23240 total trace occurrences
Log has 2 unique traces
Sample traces: [('O_Returned',), ('O_Returned', 'O_Returned')]
Unique activities: {'O_Returned'}
Base case: Single activity 'O_Returned' detected
Graph created with 3 nodes and 3 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 7 ===
__
Log has 17228 total trace occurrences
Log has 1 uniqu

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warning

No special structure found, returning single activity leaf node

=== Recursion Depth 5 ===
__
Log has 42995 total trace occurrences
Log has 1 unique traces
Sample traces: [('O_Create Offer',)]
Unique activities: {'O_Create Offer'}
Base case: Single activity 'O_Create Offer' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 4 ===
__
Log has 3753 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Denied',)]
Unique activities: {'A_Denied'}
Base case: Single activity 'A_Denied' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 3 ===
__
Log has 3720 total trace occurrences
Log has 8 unique traces
Sample traces: [('

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


In [20]:
start_for_stats_06 = time.time()
process_tree_exp_06 = recursion_full_improved(clean_log_counter, sup =0.6)
end_for_stats_06 = time.time()



=== Recursion Depth 0 ===
__
Log has 31509 total trace occurrences
Log has 2630 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'A_Denied', 'O_Refused'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Cancelled', 'O_Sent (mail and online)', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending')]
Unique activities: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Applicatio

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending',

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Submitted', 'A_Concept'}, Sigma_2={'A_Accepted'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'A_Accepted'}
Split results:
  Log1: 2 unique traces, 31509 total occurrences
  Log2: 1 unique traces, 31509 total occurrences

=== Recursion Depth 2 ===
__
Log has 31509 total trace occurrences
Log has 2 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept'), ('A_Create Application', 'A_Concept')]
Unique activities: {'A_Create Application', 

...

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warning

No special structure found, returning single activity leaf node

=== Recursion Depth 10 ===
__
Log has 31362 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Complete',)]
Unique activities: {'A_Complete'}
Base case: Single activity 'A_Complete' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 8 ===
__
Log has 23240 total trace occurrences
Log has 2 unique traces
Sample traces: [('O_Returned',), ('O_Returned', 'O_Returned')]
Unique activities: {'O_Returned'}
Base case: Single activity 'O_Returned' detected
Graph created with 3 nodes and 3 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 7 ===
__
Log has 17228 total trace occurrences
Log has 1 uniqu

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warning

No special structure found, returning single activity leaf node

=== Recursion Depth 5 ===
__
Log has 42995 total trace occurrences
Log has 1 unique traces
Sample traces: [('O_Create Offer',)]
Unique activities: {'O_Create Offer'}
Base case: Single activity 'O_Create Offer' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 4 ===
__
Log has 3753 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Denied',)]
Unique activities: {'A_Denied'}
Base case: Single activity 'A_Denied' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 3 ===
__
Log has 3720 total trace occurrences
Log has 8 unique traces
Sample traces: [('

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


In [21]:
start_for_stats_07 = time.time()
process_tree_exp_07 = recursion_full_improved(clean_log_counter, sup =0.7)
end_for_stats_07 = time.time()



=== Recursion Depth 0 ===
__
Log has 31509 total trace occurrences
Log has 2630 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'A_Denied', 'O_Refused'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Cancelled', 'O_Sent (mail and online)', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending')]
Unique activities: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Applicatio

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending',

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Submitted', 'A_Concept'}, Sigma_2={'A_Accepted'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'A_Accepted'}
Split results:
  Log1: 2 unique traces, 31509 total occurrences
  Log2: 1 unique traces, 31509 total occurrences

=== Recursion Depth 2 ===
__
Log has 31509 total trace occurrences
Log has 2 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept'), ('A_Create Application', 'A_Concept')]
Unique activities: {'A_Create Application', 

...

No special structure found, returning single activity leaf node

=== Recursion Depth 7 ===
__
Log has 42995 total trace occurrences
Log has 1 unique traces
Sample traces: [('O_Create Offer',)]
Unique activities: {'O_Create Offer'}
Base case: Single activity 'O_Create Offer' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 6 ===
__
Log has 46216 total trace occurrences
Log has 2 unique traces
Sample traces: [(), ('A_Denied',)]
Unique activities: {'A_Denied'}
Base case: Single activity 'A_Denied' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 5 ===
__
Log has 10431 total trace occurrences
Log has 1 unique traces
Sample trace

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


No special structure found, returning single activity leaf node

=== Recursion Depth 4 ===
__
Log has 17228 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Pending',)]
Unique activities: {'A_Pending'}
Base case: Single activity 'A_Pending' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 3 ===
__
Log has 3720 total trace occurrences
Log has 8 unique traces
Sample traces: [('O_Refused',), ('O_Refused', 'O_Refused', 'O_Refused'), ('O_Refused', 'O_Refused')]
Unique activities: {'O_Refused'}
Base case: Single activity 'O_Refused' detected
Graph created with 3 nodes and 3 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 2 ===
__
Log has 17228 total tr

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


No special structure found, returning single activity leaf node


In [22]:
start_for_stats_08 = time.time()
process_tree_exp_08 = recursion_full_improved(clean_log_counter, sup =0.8)
end_for_stats_08 = time.time()



=== Recursion Depth 0 ===
__
Log has 31509 total trace occurrences
Log has 2630 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'A_Denied', 'O_Refused'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Cancelled', 'O_Sent (mail and online)', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending')]
Unique activities: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Applicatio

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending',

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Submitted', 'A_Concept'}, Sigma_2={'A_Accepted'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'A_Accepted'}
Split results:
  Log1: 2 unique traces, 31509 total occurrences
  Log2: 1 unique traces, 31509 total occurrences

=== Recursion Depth 2 ===
__
Log has 31509 total trace occurrences
Log has 2 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept'), ('A_Create Application', 'A_Concept')]
Unique activities: {'A_Create Application', 

...

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warning

No special structure found, returning single activity leaf node

=== Recursion Depth 10 ===
__
Log has 31362 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Complete',)]
Unique activities: {'A_Complete'}
Base case: Single activity 'A_Complete' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 8 ===
__
Log has 23240 total trace occurrences
Log has 2 unique traces
Sample traces: [('O_Returned',), ('O_Returned', 'O_Returned')]
Unique activities: {'O_Returned'}
Base case: Single activity 'O_Returned' detected
Graph created with 3 nodes and 3 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 7 ===
__
Log has 17228 total trace occurrences
Log has 1 uniqu

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warning

No special structure found, returning single activity leaf node

=== Recursion Depth 5 ===
__
Log has 42995 total trace occurrences
Log has 1 unique traces
Sample traces: [('O_Create Offer',)]
Unique activities: {'O_Create Offer'}
Base case: Single activity 'O_Create Offer' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 4 ===
__
Log has 3753 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Denied',)]
Unique activities: {'A_Denied'}
Base case: Single activity 'A_Denied' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 3 ===
__
Log has 3720 total trace occurrences
Log has 8 unique traces
Sample traces: [('

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


In [23]:
start_for_stats_09 = time.time()
process_tree_exp_09 = recursion_full_improved(clean_log_counter, sup =0.9)
end_for_stats_09 = time.time()



=== Recursion Depth 0 ===
__
Log has 31509 total trace occurrences
Log has 2630 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'A_Denied', 'O_Refused'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Cancelled', 'O_Sent (mail and online)', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending')]
Unique activities: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Applicatio

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending',

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Submitted', 'A_Concept'}, Sigma_2={'A_Accepted'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'A_Accepted'}
Split results:
  Log1: 2 unique traces, 31509 total occurrences
  Log2: 1 unique traces, 31509 total occurrences

=== Recursion Depth 2 ===
__
Log has 31509 total trace occurrences
Log has 2 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept'), ('A_Create Application', 'A_Concept')]
Unique activities: {'A_Create Application', 

...

No special structure found, returning single activity leaf node

=== Recursion Depth 10 ===
__
Log has 31362 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Complete',)]
Unique activities: {'A_Complete'}
Base case: Single activity 'A_Complete' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 8 ===
__
Log has 23240 total trace occurrences
Log has 2 unique traces
Sample traces: [('O_Returned',), ('O_Returned', 'O_Returned')]
Unique activities: {'O_Returned'}
Base case: Single activity 'O_Returned' detected
Graph created with 3 nodes and 3 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 7 ===
__
Log has 42995 total trace occurrences
Log has 1 uniqu

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


No special structure found, returning single activity leaf node

=== Recursion Depth 5 ===
__
Log has 10431 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Cancelled',)]
Unique activities: {'A_Cancelled'}
Base case: Single activity 'A_Cancelled' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 4 ===
__
Log has 3753 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Denied',)]
Unique activities: {'A_Denied'}
Base case: Single activity 'A_Denied' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 3 ===
__
Log has 3720 total trace occurrences
Log has 8 unique traces
Sample traces: [('O_Refused

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


No special structure found, returning single activity leaf node

=== Recursion Depth 2 ===
__
Log has 17228 total trace occurrences
Log has 1 unique traces
Sample traces: [('O_Accepted',)]
Unique activities: {'O_Accepted'}
Base case: Single activity 'O_Accepted' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node


In [24]:
start_for_stats_1 = time.time()
process_tree_exp_1 = recursion_full_improved(clean_log_counter, sup =1)
end_for_stats_1 = time.time()



=== Recursion Depth 0 ===
__
Log has 31509 total trace occurrences
Log has 2630 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'A_Validating', 'A_Denied', 'O_Refused'), ('A_Create Application', 'A_Submitted', 'A_Concept', 'A_Accepted', 'O_Create Offer', 'O_Created', 'O_Sent (mail and online)', 'A_Complete', 'O_Create Offer', 'O_Created', 'O_Cancelled', 'O_Sent (mail and online)', 'A_Validating', 'O_Returned', 'A_Incomplete', 'A_Validating', 'O_Accepted', 'A_Pending')]
Unique activities: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Sent (mail and online)', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Create Applicatio

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}, Sigma_2={'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending', 'A_Incomplete', 'O_Accepted', 'O_Create Offer', 'A_Cancelled', 'O_Sent (mail and online)'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Accepted', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'O_Refused', 'A_Denied', 'O_Cancelled', 'O_Created', 'O_Returned', 'A_Complete', 'O_Sent (online only)', 'A_Validating', 'A_Pending',

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


  exc cut result: Sigma_1=set(), Sigma_2=set(), cost=None
  ✗ exc cut invalid - empty sets or infinite cost
Trying seq cut...
nodes? 
maybe this one is the problem
  seq cut result: Sigma_1={'A_Create Application', 'A_Submitted', 'A_Concept'}, Sigma_2={'A_Accepted'}, cost=0
  ✓ seq cut successful with cost 0 (new best)
  🎯 PERFECT SEQ CUT FOUND! Stopping cut evaluation for this subgraph.
  Skipping par cut - already found perfect solution
  Skipping loop cut - already found perfect solution
Best cut: seq with cost 0
Splitting log with seq cut...
  Sigma_1: {'A_Create Application', 'A_Submitted', 'A_Concept'}
  Sigma_2: {'A_Accepted'}
Split results:
  Log1: 2 unique traces, 31509 total occurrences
  Log2: 1 unique traces, 31509 total occurrences

=== Recursion Depth 2 ===
__
Log has 31509 total trace occurrences
Log has 2 unique traces
Sample traces: [('A_Create Application', 'A_Submitted', 'A_Concept'), ('A_Create Application', 'A_Concept')]
Unique activities: {'A_Create Application', 

...

No special structure found, returning single activity leaf node

=== Recursion Depth 10 ===
__
Log has 31362 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Complete',)]
Unique activities: {'A_Complete'}
Base case: Single activity 'A_Complete' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 8 ===
__
Log has 23240 total trace occurrences
Log has 2 unique traces
Sample traces: [('O_Returned',), ('O_Returned', 'O_Returned')]
Unique activities: {'O_Returned'}
Base case: Single activity 'O_Returned' detected
Graph created with 3 nodes and 3 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 7 ===
__
Log has 3753 total trace occurrences
Log has 1 unique

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


No special structure found, returning single activity leaf node

=== Recursion Depth 5 ===
__
Log has 10431 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Cancelled',)]
Unique activities: {'A_Cancelled'}
Base case: Single activity 'A_Cancelled' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 4 ===
__
Log has 17228 total trace occurrences
Log has 1 unique traces
Sample traces: [('A_Pending',)]
Unique activities: {'A_Pending'}
Base case: Single activity 'A_Pending' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node

=== Recursion Depth 3 ===
__
Log has 3720 total trace occurrences
Log has 8 unique traces
Sample traces: [('O_Ref

/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')
/Users/mish.vans/opt/anaconda3/envs/IMr/lib/python3.11/site-packages/pulp/apis/gurobi_api.py:317: UserWarning: GUROBI_CMD does provide good solution status of non optimal solutions
  warnings.warn('GUROBI_CMD does provide good solution status of non optimal solutions')


No special structure found, returning single activity leaf node

=== Recursion Depth 2 ===
__
Log has 17228 total trace occurrences
Log has 1 unique traces
Sample traces: [('O_Accepted',)]
Unique activities: {'O_Accepted'}
Base case: Single activity 'O_Accepted' detected
Graph created with 3 nodes and 2 edges
Checking for τ-xor structure using xor_cut_tau...
Checking for loop structure using loop_cut_ilp...
No special structure found, returning single activity leaf node


In [25]:
tree_data_exp_0 = process_tree_exp_0.to_dict()
tree_data_exp_01 = process_tree_exp_01.to_dict()
tree_data_exp_02 = process_tree_exp_02.to_dict()
tree_data_exp_03 = process_tree_exp_03.to_dict()
tree_data_exp_04 = process_tree_exp_04.to_dict()
tree_data_exp_05 = process_tree_exp_05.to_dict()
tree_data_exp_06 = process_tree_exp_06.to_dict()
tree_data_exp_07 = process_tree_exp_07.to_dict()
tree_data_exp_08 = process_tree_exp_08.to_dict()
tree_data_exp_09 = process_tree_exp_09.to_dict()
tree_data_exp_1 = process_tree_exp_1.to_dict()

print("tree for sup = 0: ", tree_data_exp_0, "time-taken: ", end_for_stats_0 - start_for_stats_0)
print("tree for sup = 0.1: ", tree_data_exp_01, "time-taken: ", end_for_stats_01 - start_for_stats_01)
print("tree for sup = 0.2: ", tree_data_exp_02, "time-taken: ", end_for_stats_02 - start_for_stats_02)
print("tree for sup = 0.3: ", tree_data_exp_03, "time-taken: ", end_for_stats_03 - start_for_stats_03)
print("tree for sup = 0.4: ", tree_data_exp_04, "time-taken: ", end_for_stats_04 - start_for_stats_04)
print("tree for sup = 0.5: ", tree_data_exp_05, "time-taken: ", end_for_stats_05 - start_for_stats_05)
print("tree for sup = 0.6: ", tree_data_exp_06, "time-taken: ", end_for_stats_06 - start_for_stats_06)
print("tree for sup = 0.7: ", tree_data_exp_07, "time-taken: ", end_for_stats_07 - start_for_stats_07)
print("tree for sup = 0.8: ", tree_data_exp_08, "time-taken: ", end_for_stats_08 - start_for_stats_08)
print("tree for sup = 0.9: ", tree_data_exp_09, "time-taken: ", end_for_stats_09 - start_for_stats_09)
print("tree for sup = 1: ", tree_data_exp_1, "time-taken: ", end_for_stats_1 - start_for_stats_1)

tree for sup = 0:  {'operator': 'seq', 'children': [{'operator': 'seq', 'children': [{'operator': 'seq', 'children': [{'operator': 'seq', 'children': [{'operator': 'seq', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'operator': 'exc', 'children': [{'activity': 'A_Create Application'}, {'operator': 'exc', 'children': []}]}, {'activity': 'tau'}]}, {'activity': 'tau'}]}, {'activity': 'tau'}]}, {'activity': 'tau'}]}, {'activity': 'tau'}]}, {'activity': 'tau'}]}, {'activity': 'tau'}]}, {'activity': 'tau'}]}, {'activity': 'tau'}]}, {'ac

In [26]:
from pm4py.objects.process_tree.obj import ProcessTree
from pm4py.objects.process_tree.utils import generic
from pm4py.objects.process_tree.obj import Operator

def to_pm4py_tree(node, parent=None):
    tree = ProcessTree()
    tree.parent = parent
    
    if node.activity:
        # Leaf node
        tree.label = node.activity
    else:
        # Operator node
        op_map = {
            'seq': Operator.SEQUENCE,
            'exc': Operator.XOR,
            'par': Operator.PARALLEL,
            'loop': Operator.LOOP
        }
        tree.operator = op_map.get(node.operator, None)
        for child in node.children:
            child_tree = to_pm4py_tree(child, parent=tree)
            tree.children.append(child_tree)
    return tree



In [27]:
tree_data_pm4py_exp_0 = to_pm4py_tree(process_tree_exp_0)
tree_data_pm4py_exp_01 = to_pm4py_tree(process_tree_exp_01)
tree_data_pm4py_exp_02 = to_pm4py_tree(process_tree_exp_02)
tree_data_pm4py_exp_03 = to_pm4py_tree(process_tree_exp_03)
tree_data_pm4py_exp_04 = to_pm4py_tree(process_tree_exp_04)
tree_data_pm4py_exp_05 = to_pm4py_tree(process_tree_exp_05)
tree_data_pm4py_exp_06 = to_pm4py_tree(process_tree_exp_06)
tree_data_pm4py_exp_07 = to_pm4py_tree(process_tree_exp_07)
tree_data_pm4py_exp_08 = to_pm4py_tree(process_tree_exp_08)
tree_data_pm4py_exp_09 = to_pm4py_tree(process_tree_exp_09)
tree_data_pm4py_exp_1 = to_pm4py_tree(process_tree_exp_1)

#net, im, fm = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py)
net_exp_0, im_exp_0, fm_exp_0 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_0)
net_exp_01, im_exp_01, fm_exp_01 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_01)
net_exp_02, im_exp_02, fm_exp_02 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_02)
net_exp_03, im_exp_03, fm_exp_03 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_03)
net_exp_04, im_exp_04, fm_exp_04 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_04)
net_exp_05, im_exp_05, fm_exp_05 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_05)
net_exp_06, im_exp_06, fm_exp_06 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_06)
net_exp_07, im_exp_07, fm_exp_07 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_07)
net_exp_08, im_exp_08, fm_exp_08 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_08)
net_exp_09, im_exp_09, fm_exp_09 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_09)
net_exp_1, im_exp_1, fm_exp_1 = pm4py.objects.conversion.process_tree.converter.apply(tree_data_pm4py_exp_1)

In [28]:
import matplotlib.pyplot as plt
from pm4py.visualization.petri_net import visualizer as pn_visualizer
import tempfile
from PIL import Image
import os

def save_multiple_petri_nets_to_pdf(petri_nets_list, filename="multiple_petri_nets.pdf"):
    """
    Save multiple Petri nets to a single PDF
    petri_nets_list: List of tuples [(net1, im1, fm1), (net2, im2, fm2), ...]
    """
    images = []
    
    # Create temporary images for each Petri net
    for i, (net, im, fm) in enumerate(petri_nets_list):
        # Generate visualization
        gviz = pn_visualizer.apply(net, im, fm)
        
        # Save as temporary PNG
        temp_path = f"temp_petri_net_{i}.png"
        pn_visualizer.save(gviz, temp_path)
        images.append(temp_path)
    
    # Convert images to PDF
    first_image = Image.open(images[0])
    other_images = [Image.open(img) for img in images[1:]]
    
    first_image.save(filename, "PDF", resolution=100.0, save_all=True, 
                    append_images=other_images)
    
    # Clean up temporary files
    for img_path in images:
        os.remove(img_path)
    
    print(f"Saved {len(petri_nets_list)} Petri nets to {filename}")



In [29]:
petri_nets = [
    (net_exp_0, im_exp_0, fm_exp_0),
    (net_exp_01, im_exp_01, fm_exp_01),
    (net_exp_02, im_exp_02, fm_exp_02),
    (net_exp_03, im_exp_03, fm_exp_03),
    (net_exp_04, im_exp_04, fm_exp_04),
    (net_exp_05, im_exp_05, fm_exp_05),
    (net_exp_06, im_exp_06, fm_exp_06),
    (net_exp_07, im_exp_07, fm_exp_07),
    (net_exp_08, im_exp_08, fm_exp_08),
    (net_exp_09, im_exp_09, fm_exp_09),
    (net_exp_1, im_exp_1, fm_exp_1)
]

save_multiple_petri_nets_to_pdf(petri_nets, "IM_ilp/models_2017_try.pdf")

Saved 11 Petri nets to IM_ilp/models_2017_try.pdf
